# Day 19
## Part 1
~~I'm not bothering with this one.~~ Ok, the single empty day is nagging at me. Let's take this one slowly and not spend entire days debugging it.

Parse the scanners as lists of three-coordinate tuples.

In [1]:
import parse

def parse_data(s):
    scanners = []
    for block in s.strip().split('\n\n'):
        scanner = []
        for line in block.splitlines():
            if r := parse.parse('{x:d},{y:d},{z:d}', line):
                scanner.append((r['x'], r['y'], r['z']))
        scanners.append(scanner)
    return scanners

test_data = parse_data(open('test_input', 'r').read())
data = parse_data(open('input', 'r').read())

Record the distance between each scanner's observations as a set of absolute differences in each coordinate direction, which will allow comparison with other scanners' coordinate systems.

In [8]:
import itertools
import networkx as nx

def beacon_distance(scanner):
    distances = nx.Graph()
    for (i, (x_i, y_i, z_i)), (j, (x_j, y_j, z_j)) in itertools.combinations(enumerate(scanner), 2):
        distance = frozenset([abs(d) for d in [x_i - x_j, y_i - y_j, z_i - z_j]])
        distances.add_edge(i, j, distance=distance)
    return distances

beacon_distances = [beacon_distance(x) for x in test_data]

In [14]:
beacon_distances[0][0][1]

{'distance': frozenset({55, 124, 1310})}

In [15]:
beacon_distances[0][1][0]

{'distance': frozenset({55, 124, 1310})}